# Features - virtual time series - DEV

This code processes all variables at once. 

Advantage: The QA layer has to be loaded only once

Disadvantage: Parallelization and check for / skip over esults of existing single bands with Snakemake. 

**TODO**: Create a Snakemake task.

In [108]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# - - - - - - - - - - - - - - - - - - - - 
# DEFAULT IMPORTS - IN ALL NOTEBOKS
from src import configs

prjconf = configs.ProjectConfigParser()

# - - - - - - - - - - - - - - - - - - - - 
# NOTEBOOK SPECIFIC IMPORTS
import numpy as np
from pathlib import Path
import pandas as pd
import rasterio
from tqdm import tqdm

from eobox.raster import cube
from eobox.raster import gdalutils

tilenames = prjconf.get("Params", "tiles").split(" ")
tilenames

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


['32UNU', '32UPU', '32UQU', '33UUP']

## Inputs

### Parameters for the script

In [109]:
scoll_id = 1
scoll_name = f"scoll{scoll_id:02d}"

variables = ["Red", "NIR", "SWIR1", "SWIR2"]
qa = "CLEAR"
qa_valid = [1]

idx_virtual = pd.date_range(start='2018-01-01', end="2018-12-31", freq='4W')


print(f"{'scoll_name':12s}: {scoll_name}")
print(f"{'variables':12s}: {variables}")
print(f"{'qa':12s}: {qa}")
print(f"{'idx_virtual':12s}: {idx_virtual}")


scoll_name  : scoll01
variables   : ['Red', 'NIR', 'SWIR1', 'SWIR2']
qa          : CLEAR
idx_virtual : DatetimeIndex(['2018-01-07', '2018-02-04', '2018-03-04', '2018-04-01',
               '2018-04-29', '2018-05-27', '2018-06-24', '2018-07-22',
               '2018-08-19', '2018-09-16', '2018-10-14', '2018-11-11',
               '2018-12-09'],
              dtype='datetime64[ns]', freq='4W-SUN')


### Files

In Snakemake the list would be created by wildcard catching the tiles.

In [110]:
scolls = {tile: (prjconf.get_path("Raw", "scene_colls") / tile / f"df_{scoll_name}.csv") for tile in tilenames}

## Outputs

In [112]:
dst_paths = {}
for tile in tilenames:
    dst_paths[tile] = prjconf.get_paths_features_vts_regular_raster(scoll_name, tile, variables, idx_virtual, as_dict=True)
    for var in variables:
        print(f"First and last file (of {len(dst_paths[tile][var])}) of {(tile + ' ' + var)}")
        print("   " + dst_paths[tile][var][0])
        print("   " + dst_paths[tile][var][-1])
        print()

First and last file (of 13) of 32UNU Red
   /home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UNU/scoll01/32unu__scoll01__vts4w__2018-01-07__Red.vrt
   /home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UNU/scoll01/32unu__scoll01__vts4w__2018-12-09__Red.vrt

First and last file (of 13) of 32UNU NIR
   /home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UNU/scoll01/32unu__scoll01__vts4w__2018-01-07__NIR.vrt
   /home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UNU/scoll01/32unu__scoll01__vts4w__2018-12-09__NIR.vrt

First and last file (of 13) of 32UNU SWIR1
   /home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UNU/scoll01/32unu__scoll01__vts4w__2018-01-07__SWIR1.vrt
   /home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UNU/scoll01/32unu__scoll01__vts4w__2018-12-09__SWIR1.vrt

First and last file (of 13) of 32UNU SWIR2
   /home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UNU/scoll01/32unu__

In [106]:
for tile in tilenames:
    print("*" * 100)
    print(tile)

    scoll_layers = prjconf.get_layer_df_of_scene_collection(scoll_name, variables + [qa], tile)
    assert len(scoll_layers["tile"].unique()) == 1
    assert len(scoll_layers["product"].unique()) == 1  
    # here L30 & S30 would also make sense but then we need to change the product string below
    # tile = scoll_layers["tile"].unique()[0]
    # product = scoll_layers["product"].unique()[0]

    dst_pattern = prjconf.get_paths_features_vts_regular_raster(scoll_name, tile, variables, idx_virtual, as_dict=True, return_patter=True)

    print(dst_pattern)

    ## Run Task

    scoll = cube.EOCubeSceneCollection(df_layers=scoll_layers, 
                                       chunksize=2**9, 
                                       variables=variables, 
                                       qa=qa, 
                                       qa_valid=qa_valid 
                                      )

    scoll.create_virtual_time_series(
        idx_virtual=idx_virtual,
        dst_pattern=dst_pattern,
        dtypes="int16",
        compress='lzw',
        nodata=None,
        num_workers=4)
    # 64it [2:24:38, 25.54s/it]


    ## Create VRTs
    # Create a time series layer stack (VRT) for each variable. 

    dst_dir = prjconf.get_path("Processed", "raster", tile=tile) / scoll_name
    dst_dir_vsts_stack = prjconf.get_path("Processed", "raster", tile=tile) / "VRTs" / "ts_per_band" / tile / f"{scoll_name}"
    dst_dir_vsts_stack.mkdir(parents=True, exist_ok=True)
    dst_dir_vsts_stack

    for var in scoll.variables:
        input_file_list = list(list(Path(dst_dir).glob(f"*{var}*.vrt")))
        input_file_list = np.sort(input_file_list)
        output_file = Path(dst_dir_vsts_stack) / f"{tile.lower()}__{scoll_name}__vts4w__{var}.vrt"
        print(output_file)
        gdalutils.buildvrt(input_file_list, output_file, relative=True, separate=True)

****************************************************************************************************
32UNU
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UNU/scoll01/32unu__scoll01__vts4w__{date}__{var}.vrt
Red: 64 / 64 chunks already processed and skipped.
NIR: 64 / 64 chunks already processed and skipped.


  0%|          | 0/4 [00:00<?, ?it/s]

SWIR1: 64 / 64 chunks already processed and skipped.
SWIR2: 64 / 64 chunks already processed and skipped.
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UNU/VRTs/ts_per_band/32UNU/scoll01/32unu__scoll01__vts4w__Red.vrt
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UNU/VRTs/ts_per_band/32UNU/scoll01/32unu__scoll01__vts4w__NIR.vrt
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UNU/VRTs/ts_per_band/32UNU/scoll01/32unu__scoll01__vts4w__SWIR1.vrt
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UNU/VRTs/ts_per_band/32UNU/scoll01/32unu__scoll01__vts4w__SWIR2.vrt
****************************************************************************************************
32UPU
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UPU/scoll01/32upu__scoll01__vts4w__{date}__{var}.vrt



  0%|          | 0/4 [00:00<?, ?it/s]

Red: 64 / 64 chunks already processed and skipped.
NIR: 64 / 64 chunks already processed and skipped.
SWIR1: 64 / 64 chunks already processed and skipped.
SWIR2: 64 / 64 chunks already processed and skipped.
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UPU/VRTs/ts_per_band/32UPU/scoll01/32upu__scoll01__vts4w__Red.vrt
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UPU/VRTs/ts_per_band/32UPU/scoll01/32upu__scoll01__vts4w__NIR.vrt
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UPU/VRTs/ts_per_band/32UPU/scoll01/32upu__scoll01__vts4w__SWIR1.vrt
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UPU/VRTs/ts_per_band/32UPU/scoll01/32upu__scoll01__vts4w__SWIR2.vrt
****************************************************************************************************
32UQU
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UQU/scoll01/32uqu__scoll01__vts4w__{date}__{var}.vrt



  0%|          | 0/4 [00:00<?, ?it/s]

Red: 64 / 64 chunks already processed and skipped.
NIR: 64 / 64 chunks already processed and skipped.
SWIR1: 64 / 64 chunks already processed and skipped.
SWIR2: 64 / 64 chunks already processed and skipped.
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UQU/VRTs/ts_per_band/32UQU/scoll01/32uqu__scoll01__vts4w__Red.vrt
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UQU/VRTs/ts_per_band/32UQU/scoll01/32uqu__scoll01__vts4w__NIR.vrt
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UQU/VRTs/ts_per_band/32UQU/scoll01/32uqu__scoll01__vts4w__SWIR1.vrt
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/32UQU/VRTs/ts_per_band/32UQU/scoll01/32uqu__scoll01__vts4w__SWIR2.vrt
****************************************************************************************************
33UUP
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/33UUP/scoll01/33uup__scoll01__vts4w__{date}__{var}.vrt



  0%|          | 0/4 [00:00<?, ?it/s]

Red: 64 / 64 chunks already processed and skipped.
NIR: 64 / 64 chunks already processed and skipped.
SWIR1: 64 / 64 chunks already processed and skipped.
SWIR2: 64 / 64 chunks already processed and skipped.
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/33UUP/VRTs/ts_per_band/33UUP/scoll01/33uup__scoll01__vts4w__Red.vrt
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/33UUP/VRTs/ts_per_band/33UUP/scoll01/33uup__scoll01__vts4w__NIR.vrt
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/33UUP/VRTs/ts_per_band/33UUP/scoll01/33uup__scoll01__vts4w__SWIR1.vrt
/home/ben/Devel/Projects/classify-hls/data/processed/L3/raster/33UUP/VRTs/ts_per_band/33UUP/scoll01/33uup__scoll01__vts4w__SWIR2.vrt
